In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tslearn.clustering import TimeSeriesKMeans
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
import pandas as pd
from fastdtw import fastdtw

In [ ]:
df1 = pd.read_csv('/Users/sponnur/PycharmProjects/pythonProject1/cpu_usage_2022_03_25_35_15m.csv')
#df2 = pd.read_csv('/Users/sponnur/PycharmProjects/pythonProject1/mem_usage_2022_04_27_2_15m.csv')
#df3 = pd.read_csv('/Users/sponnur/PycharmProjects/pythonProject1/api_server_count_2022_04_27_2_15m.csv')

In [ ]:
df1.timestamp = pd.to_datetime(df1.timestamp, infer_datetime_format=True)  

In [ ]:
cluster_id_list = df1['cluster_id'].to_list()

In [ ]:
df1.set_index('timestamp',inplace=True)

In [ ]:
sample_cluster_df = df1[df1.cluster_id.isin(list(set(cluster_id_list))[:10000])]

In [ ]:
sample_cluster_df['date'] = sample_cluster_df.index.date

In [ ]:
len(set(sample_cluster_df['cluster_id'].to_list()))

In [ ]:
single_day_df = sample_cluster_df.loc[(sample_cluster_df.index.floor('D') == '2022-03-26')]

In [ ]:
two_day_df = df2.loc[(df2.index.floor('D') == '2022-03-26') | (df2.index.floor('D') == '2022-03-27')]

In [ ]:
df_single_cluster = single_day_df.loc[single_day_df['cluster_id'] == '01d1e513-9210-4099-967f-6dc13a7b4cc4']

# Total Avg of Cpu
# Min of the cluster cpu usage
# Max of the cluster cpu usage
# Top 5 Min AVG cpu usage
# Top 5 Max AVG cpu usage

In [ ]:
def generate_features(current_list):
    #print(current_list)
    current_list.sort()
    metric_max = max(current_list)
    metric_min = min(current_list)
    metric_avg = sum(current_list)/len(current_list)
    top_5_max_avg = sum(current_list[:5])/5
    top_5_min_avg = sum(current_list[-5:])/5
    min_max_diff = metric_max - metric_min
    min_max_5_diff = top_5_max_avg - top_5_min_avg
    #print(metric_max, metric_min, metric_avg, top_5_max_avg, top_5_min_avg, min_max_diff, min_max_5_diff)
    return [ metric_max, metric_min, metric_avg, top_5_max_avg, top_5_min_avg, min_max_diff, min_max_5_diff]
    

In [ ]:
sample_cluster_df.head()

In [ ]:
#single day
new_list = []
for cluster_id, current_df1 in single_day.groupby('cluster_id'):
    for date, current_df2 in current_df1.groupby('date'):
        current_list = current_df2['cpu_usage'].to_list()
        features_generated = [cluster_id]
        features_generated.extend(generate_features(current_list))
        new_list.append(features_generated)
    print(cluster_id, count)
new_df = pd.DataFrame(new_list,columns=['cluster_id','metric_max', 'metric_min', 'metric_avg', 'top_5_max_avg', 'top_5_min_avg', 'min_max_diff', 'min_max_5_diff'])
print(count)

In [ ]:
#all_days
new_list = []
resampled_df = sample_cluster_df.sample(100)
for cluster_id, current_df1 in sample_cluster_df.groupby('cluster_id'):
        #print(cluster_id,current_df1.head())
        current_list = current_df1['cpu_usage'].to_list()
        features_generated = [cluster_id]
        features_generated.extend(generate_features(current_list))
        new_list.append(features_generated)
new_df2 = pd.DataFrame(new_list,columns=['cluster_id','all_metric_max', 'all_metric_min', 'all_metric_avg', 'all_top_5_max_avg', 'all_top_5_min_avg', 'all_min_max_diff', 'all_min_max_5_diff'])

In [ ]:
from scipy.stats import zscore

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
db = DBSCAN(eps=35,min_samples=10).fit(new_df2[['all_metric_max', 'all_metric_min', 'all_metric_avg', 'all_top_5_max_avg', 'all_top_5_min_avg', 'all_min_max_diff', 'all_min_max_5_diff']])

In [6]:
from collections import Counter
import pickle

In [ ]:
labels = db.labels_

In [5]:
Counter(labels)

NameError: name 'labels' is not defined

In [ ]:
with open('current_labels.pkl','wb+') as f:
    pickle.dump(labels,f)

In [7]:
with open('current_labels.pkl','rb') as f:
    new_list2 = pickle.load(f)

In [8]:
Counter(new_list2)

Counter({0: 9924, -1: 76})